In [ ]:
https://tykimos.github.io/2017/08/07/Keras_Install_on_Windows/
    
    # 김태영님 캐글콜리아.
    => 프로젝트 디렉토리 만듦, 가상환경 만들어서 가상환경 실행하기
    # 수업때 진행했던 것처럼. 가상환경 venv 만들었고
    # 여기다가  주요패키지 설치할것
    # 딥러닝 라이브러리 설치 => 텐서플로우 2.1 버전 업그레이드
    
    # 케라스 설치 2.3.1
    
    # 케라스 구동 API 가 다 틀려서 결국 텐서플로우1.14 버전으로 실행중
    아래 소스는 텐서플로우 1버전의 케라스 구동API
    텐서플로우 2.0 버전에서 소스뜯다가 실패

In [5]:
import os, glob, numpy as np
import tensorflow.keras as keras
# module 'tensorflow' has no attribute 'get_default_graph'

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy


import matplotlib.pyplot as plt
import keras.backend.tensorflow_backend as K
from tensorflow.python.framework import ops
ops.reset_default_graph()
import tensorflow as tf
'''
config = tf.ConfigProto()  텐서플로우 2. 에서 구동안됨
module 'tensorflow' has no attribute 'ConfigProto'
'''
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True

# session = tf.Session(config=config) 자동세션기능
X_train, X_test, y_train, y_test = np.load('./multi_image_data.npy',allow_pickle=True)
print(X_train.shape)
print(X_train.shape[0])

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


categories = ["corgi", "golden" ,"dach" ,"husky" ,"beagle"]
nb_classes = len(categories)

#일반화
X_train = X_train.astype(float) / 255
X_test = X_test.astype(float) / 255


(3014, 100, 100, 3)
3014
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17016229528493769802
]


In [ ]:
모델 평가
앞으로 네트워크를 만들 때 배치 정규화는 웬만하면 적용해 주는 것이 좋다
(논문에 따르면 뒤에서 나올 regularization의 효과도 있어 배치 정규화를 적용하면 
 드랍아웃을 적용하지 않아도 된다고 한다).

https://buomsoo-kim.github.io/keras/2018/04/24/Easy-deep-learning-with-Keras-5.md/

In [7]:
# 배치정규화 시켜서 CNN 모델을 만들자

def cnn_model(): # create model 
    model = Sequential() 
    
    model.add(Conv2D(16, 3, 3, border_mode='same', activation='relu',
                            input_shape=X_train.shape[1:]))
    model.add(BatchNormalization()) 
    model.add(MaxPooling2D(pool_size=(2, 2))) 
    
    model.add(Conv2D(32, 3, 3, border_mode='same', activation='relu')) 
    model.add(BatchNormalization()) 
    model.add(MaxPooling2D(pool_size=(2, 2))) 
    
    model.add(Conv2D(32, 3, 3, border_mode='same', activation='relu')) 
    model.add(BatchNormalization()) 
    model.add(MaxPooling2D(pool_size=(2, 2))) 
    
    model.add(Conv2D(64, 3, 3, border_mode='same', activation='relu')) 
    model.add(MaxPooling2D(pool_size=(2, 2))) 
    
    model.add(Conv2D(64, 3, 3, border_mode='same', activation='relu')) 
    model.add(BatchNormalization()) 
    model.add(MaxPooling2D(pool_size=(2, 2))) 
    
    
    model.add(Flatten()) 
    model.add(Dense(64, activation='relu')) 
    model.add(Dropout(0.5)) 
    model.add(Dense(64, activation='relu')) 
    model.add(Dropout(0.5)) 
    model.add(Dense(5, activation='softmax')) 
    return model

model2= cnn_model()
model2.summary()

C:\Users\ICT01_02\Anaconda3\envs\tf_test\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu", input_shape=(100, 100,..., padding="same")`
  """
C:\Users\ICT01_02\Anaconda3\envs\tf_test\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
  if __name__ == '__main__':
C:\Users\ICT01_02\Anaconda3\envs\tf_test\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
  del sys.path[0]
C:\Users\ICT01_02\Anaconda3\envs\tf_test\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 100, 100, 16)      448       
_________________________________________________________________
batch_normalization_5 (Batch (None, 100, 100, 16)      64        
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 50, 50, 32)        4640      
_________________________________________________________________
batch_normalization_6 (Batch (None, 50, 50, 32)        128       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 25, 25, 32)       

C:\Users\ICT01_02\Anaconda3\envs\tf_test\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`


In [13]:
def compile_and_train(model, num_epochs):
    model.compile(loss = categorical_crossentropy, optimizer=Adam(), metrics=['acc'])
    history= model.fit(x=X_train, y=y_train, batch_size=32, epochs=num_epochs, verbose=1,
                       validation_split =0.2) 

    
_ = compile_and_train(model2, num_epochs=20)


Train on 2411 samples, validate on 603 samples
Epoch 1/20
2411/2411 [==============================] - 12s 5ms/step - loss: 1.6554 - acc: 0.3073 - val_loss: 1.6303 - val_acc: 0.3134
Epoch 2/20
2411/2411 [==============================] - 11s 5ms/step - loss: 1.3146 - acc: 0.4152 - val_loss: 1.8757 - val_acc: 0.2272
Epoch 3/20
2411/2411 [==============================] - 11s 5ms/step - loss: 1.1931 - acc: 0.4890 - val_loss: 1.7394 - val_acc: 0.2604
Epoch 4/20
2411/2411 [==============================] - 11s 5ms/step - loss: 1.0871 - acc: 0.5350 - val_loss: 1.3405 - val_acc: 0.4362
Epoch 5/20
2411/2411 [==============================] - 11s 5ms/step - loss: 0.9577 - acc: 0.5964 - val_loss: 0.9844 - val_acc: 0.6285
Epoch 6/20
2411/2411 [==============================] - 11s 5ms/step - loss: 0.8983 - acc: 0.6387 - val_loss: 0.8857 - val_acc: 0.6451
Epoch 7/20
2411/2411 [==============================] - 11s 5ms/step - loss: 0.8305 - acc: 0.6636 - val_loss: 0.9170 - val_acc: 0.7214
Epoch 8

In [14]:
print("정확도 : %.4f" % (model2.evaluate(X_test, y_test)[1]))

def evaluate_error(model):
    pred = model.predict(X_test, batch_size=32)
    pred = np.argmax(pred, axis=1)
    pred = np.expand_dims(pred, axis=1) # make same shape as y_test
    error = np.sum(np.not_equal(pred, y_test))/ y_test.shape[0]
    return error


evaluate_error(model2)

1005/1005 [==============================] - 2s 2ms/step
정확도 : 0.8438


3.7761194029850746